In [1]:
#!/usr/bin/env python3
"""
Ultra Simple Edge AI: Recyclable Classification
===============================================
Minimal code to train and deploy a lightweight model.

Install: pip install tensorflow numpy
"""

import tensorflow as tf
import numpy as np
import time

# Configuration
IMG_SIZE = 32
CLASSES = ['plastic', 'metal', 'glass']

def create_data():
    """Generate simple test data"""
    x = np.random.rand(300, IMG_SIZE, IMG_SIZE, 3).astype(np.float32)
    y = np.random.randint(0, 3, 300)

    # Add basic patterns
    for i in range(300):
        if y[i] == 0: x[i, :, :, 2] += 0.3    # plastic = blue
        elif y[i] == 1: x[i] *= 0.5           # metal = dark
        elif y[i] == 2: x[i] += 0.3           # glass = bright

    return np.clip(x, 0, 1), y

def create_model():
    """Simple CNN model"""
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(8, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(16, 3, activation='relu'),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

def main():
    print("Training Edge AI Model...")

    # Create data
    x, y = create_data()
    split = 240
    x_train, x_test = x[:split], x[split:]
    y_train, y_test = y[:split], y[split:]

    # Train model
    model = create_model()
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=3, verbose=1)

    # Test accuracy
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"Accuracy: {accuracy:.3f}")

    # Convert to TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    # Save model
    with open('model.tflite', 'wb') as f:
        f.write(tflite_model)

    print(f"Model saved: {len(tflite_model)/1024:.1f} KB")

    # Test speed
    interpreter = tf.lite.Interpreter(model_path='model.tflite')
    interpreter.allocate_tensors()

    start = time.time()
    for i in range(10):
        interpreter.set_tensor(0, np.expand_dims(x_test[0], 0))
        interpreter.invoke()

    avg_time = (time.time() - start) * 100  # ms per inference
    print(f"Speed: {avg_time:.1f} ms")
    print(f"Edge AI Benefits: Fast ({avg_time:.1f}ms), Private, Offline")

if __name__ == "__main__":
    main()

# Raspberry Pi Usage:
# import tflite_runtime.interpreter as tflite
# interpreter = tflite.Interpreter('model.tflite')
# interpreter.allocate_tensors()
# interpreter.set_tensor(0, image_array)
# interpreter.invoke()
# result = interpreter.get_tensor(1)

Training Edge AI Model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.3373 - loss: 1.0564
Epoch 2/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.3847 - loss: 1.0076
Epoch 3/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.3789 - loss: 0.9918
Accuracy: 0.350
Saved artifact at '/tmp/tmpd3pu1_l3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  134576160083408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134576160086672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134576160085328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134576160083792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134576160086096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134576160085712: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model saved: 5.2 KB
Speed: 0.1 ms
Ed

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
